In [ ]:
import hashlib
import json
from time import time
from urllib.parse import urlparse
from uuid import uuid4

import requests
from flask import Flask, jsonify, request

In [ ]:
class Blockchain:
    

### def__init__ :     constructor of this class
- initialize 2 empty list to store chain &   transactions

In [2]:
  def __init__(self):
        self.current_transactions = []
        self.chain = []
        self.nodes = set()

        # Create the genesis block
        self.new_block(previous_hash='1', proof=100)

 

### register_node(self, address)
-   새 node를 네트워크에 등록하기 위한 메서드, 이때, 새로운 노드가 네트워크에 등록되면,
    인접한 노드 순서대로, 다른 노드들에도 그 정보가 전달되어야 한다. 
-   :param address: Address of node. Eg. 'http://192.168.0.5:5000'

In [ ]:
       
    def register_node(self, address):
        """
        Add a new node to the list of nodes
        :param address: Address of node. Eg. 'http://192.168.0.5:5000'
        """

        parsed_url = urlparse(address)
        if parsed_url.netloc:
            self.nodes.add(parsed_url.netloc)
        elif parsed_url.path:
            # Accepts an URL without scheme like '192.168.0.5:5000'.
            self.nodes.add(parsed_url.path)
        else:
            raise ValueError('Invalid URL')

### valid_chain(self, chain)
- parameter로 받아온 체인에 대해, 체인의 모든 블록들에 대해 해쉬값과 proof값을 확인함으로서, 체인의 유효성을 검사한다.
- 분기로 인한 충돌문제가 생겼을 때, 선택된 체인에 대해 검증할 때 쓰인다.
~~~python
    # Check that the hash of the block is correct
    # param chain으로 받아온 체인의 current_index번째 블록의 이전 해쉬값과, current_index-1번째 블록의 해쉬값 비교.
    last_block_hash = self.hash(last_block)
    if block['previous_hash'] != last_block_hash:
        return False

    # Check that the Proof of Work is correct
    # 블록마다 작업증명이 제대로 이루어졌는지 확인.
    if not self.valid_proof(last_block['proof'], block['proof'], last_block_hash):
        return False
~~~

In [ ]:
    def valid_chain(self, chain):
        """
        Determine if a given blockchain is valid - 1.

        :param chain: A blockchain
        :return: True if valid, False if not
        """
       
        last_block = chain[0]
        current_index = 1

        while current_index < len(chain):
            block = chain[current_index]
            print(f'{last_block}')
            print(f'{block}')
            print("\n-----------\n")
            # Check that the hash of the block is correct
            # param chain으로 받아온 체인의 current_index번째 블록의 이전 해쉬값과, current_index-1번째 블록의 해쉬값 비교.
            last_block_hash = self.hash(last_block)
            if block['previous_hash'] != last_block_hash:
                return False

            # Check that the Proof of Work is correct
            # 블록마다 작업증명이 제대로 이루어졌는지 확인.
            if not self.valid_proof(last_block['proof'], block['proof'], last_block_hash):
                return False

            last_block = block
            current_index += 1

        return True

### resolve_conflicts(self)
- consensus algorithm구현,  체인이 분기가 이루어 져서 충돌이 일어나는 경우,네트워크에서 두 체인 중 더 긴 체인을 선택하는 방식으로 충돌을 해결한다. 

~~~python
# Check if the length is longer and the chain is valid
#(max_length - 자신의 길이 와 길이비교 && 체인의 유효성검사 - 위의 valid_chain함수를 이용)
if length > max_length and self.valid_chain(chain):
    max_length = length
    new_chain = chain
# Replace our chain if we discovered a new, valid chain longer than ours
if new_chain:
    self.chain = new_chain
    return True

return False
~~~

In [ ]:
    def resolve_conflicts(self):
        """
        This is our consensus algorithm, it resolves conflicts
        by replacing our chain with the longest one in the network.
    
        :return: True if our chain was replaced, False if not
        """
        neighbours = self.nodes
        new_chain = None

        # We're only looking for chains longer than ours
        max_length = len(self.chain)

        # Grab and verify the chains from all the nodes in our network
        for node in neighbours:
            response = requests.get(f'http://{node}/chain')

            if response.status_code == 200:
                length = response.json()['length']
                chain = response.json()['chain']

                # Check if the length is longer and the chain is valid
                #(max_length - 자신의 길이 와 길이비교 && 체인의 유효성검사 - 위의 valid_chain함수를 이용)
                if length > max_length and self.valid_chain(chain):
                    max_length = length
                    new_chain = chain

        # Replace our chain if we discovered a new, valid chain longer than ours
        if new_chain:
            self.chain = new_chain
            return True

        return False

### new_block(self, proof, previous_hash)
- 새로운 블럭을 체인에 추가한다.
- 파라미터로 받아온 `proof`, `previous_hash`값을 체인에 추가될 블럭에 정보로 추가한다.
- 블럭이 갖는 정보는 다음과 같다.
~~~
block = {
    'index': len(self.chain) + 1,
    'timestamp': time(),
    'transactions': self.current_transactions,
    'proof': proof,
    'previous_hash': previous_hash or self.hash(self.chain[-1]),
}
~~~

In [ ]:
    def new_block(self, proof, previous_hash):
        """
        Create a new Block in the Blockchain - 채굴작업 성공시:proof값 발견. 새로운 블록을 생성, 
        

        :param proof: The proof given by the Proof of Work algorithm - 발견한 새로운 proof값
        :param previous_hash: Hash of previous Block -  이전 해쉬값을 받아와서 블록헤더에 저장.
        :return: New Block
        """

        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }

        # Reset the current list of transactions
        self.current_transactions = []

        #체인에 새 블록을 추가한다.
        self.chain.append(block)
        return block
    

### new_transaction(self, sender, recipient, amount)
- '/transactions/new'로 post요청이 들어오면, 새 트랜잭션을 추가한다. new_transaction()에서 호출한다.
-  추가된 트랜잭션을 정보로 갖는 __블럭의 인덱스값__ 을 리턴한다.
-  유저가 서버로 보내는 정보 3가지
~~~python
param sender: Address of the Sender
param recipient: Address of the Recipient
param amount: Amount
~~~


In [ ]:
    '''
    '/transactions/new'로 post요청이 들어오면, 새 트랜잭션을 추가한다.
    new_transaction()에서 호출한다.
    
    '''
    def new_transaction(self, sender, recipient, amount):
        """
        Creates a new transaction to go into the next mined Block

        유저가 서버로 보내는 정보 3가지
        :param sender: Address of the Sender
        :param recipient: Address of the Recipient
        :param amount: Amount
        
        :return: The index of the Block that will hold this transaction
        """
        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
        })

        return self.last_block['index'] + 1

In [ ]:
    @property
    def last_block(self):
        return self.chain[-1]

### hash(block)
- block의 해쉬값을 구해준다. 단순히 해쉬값 계산만 해주는 함수이므로, 인스턴스에 구애받지 않도록 `staticmethod`로 선언함.

In [ ]:
    @staticmethod
    def hash(block):
        """
        Creates a SHA-256 hash of a Block

        :param block: Block
        """

        # We must make sure that the Dictionary is Ordered, or we'll have inconsistent hashes
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

### proof_of_work(self, last_block)
- 작업증명 알고리즘(noncer값 구하기 - 여기선 proof값)을 구현한 함수 `valid_proof(last_proof, proof, last_hash)`를 호출한다.


In [ ]:
    def proof_of_work(self, last_block):
        """

        Simple Proof of Work Algorithm:

         - Find a number p' such that hash(p * p') contains leading 4 zeroes
         - Where p is the previous proof, and p' is the new proof
         
        :param last_block: <dict> last Block
        :return: <int>
        """

        last_proof = last_block['proof']
        last_hash = self.hash(last_block)

        proof = 0
        ## hash값을 valid_proof함수에 넣어서, 나온 값이 작업증명을 만족하는 지 확인한다.
        while self.valid_proof(last_proof, proof, last_hash) is False:
            proof += 1

        return proof

###  valid_proof(last_proof, proof, last_hash)
- 작업증명 알고리즘을 적용시킨 함수. 여기서 쓰인 작업증명 알고리즘은 다음과 같다.
>  - Find a number p' such that hash(p * p') contains leading 4 zeroes
   Where p is the previous proof, and p' is the new proof
   
 ~~~python
        #참고 - 파이썬 3.6에서 추가된 f-string이라는 문법으로 format string guess를 생성한다.
        guess = f'{last_proof}{proof}{last_hash}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        #앞의 4개의 숫자가 0000이 되는경우, true
        return guess_hash[:4] == "0000"
~~~


In [ ]:
    @staticmethod
    def valid_proof(last_proof, proof, last_hash):
        
        '''
        작업증명 알고리즘을 적용시킨 함수.
        '''
        """
        Validates the Proof

        :param last_proof: <int> Previous Proof
        :param proof: <int> Current Proof
        :param last_hash: <str> The hash of the Previous Block
        :return: <bool> True if correct, False if not.

        """
        #참고 - 파이썬 3.6에서 추가된 f-string이라는 문법으로 format string guess를 생성한다.
        guess = f'{last_proof}{proof}{last_hash}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        #앞의 4개의 숫자가 0000이 되는경우, true
        return guess_hash[:4] == "0000"

### 아래부터는 Flask를 사용해서 파이썬 웹 어플리케이션을 구축하는 과정이다.
### 인스턴스를 추가해주고, `app.route()`로 요청주소(와 요청방식)와 callback메소드를 엮어준다

In [ ]:
# Instantiate the Node
app = Flask(__name__)

# Generate a globally unique address for this node
node_identifier = str(uuid4()).replace('-', '')

# Instantiate the Blockchain
blockchain = Blockchain()



#이 아래로는 전부 url을 통해 요청을 하면, 그에 해당하는 callback함수들이 정의되어있다, callback함수들은 요청에 해당하는 작업들을
#위에서 정의한 함수들을 이용해 처리한다.

#### @app.route('/mine', methods=['GET']) def mine()
- `/mine` 주소로 요청하게 되면, 채굴작업을 하라는 의미이다.
- 채굴에 성공한 경우, 추가되는 transaction에서, sender는 "0"으로 정의한다.
- 보상은 1코인으로 하자.
- 성공시, 응답으로 블록의 index(몇 번째 블록인가), transaction, proof값, 이전해쉬를 포함한 json응답과 200코드를 response한다.



In [ ]:
@app.route('/mine', methods=['GET'])


def mine():
    # We run the proof of work algorithm to get the next proof...
    last_block = blockchain.last_block
    proof = blockchain.proof_of_work(last_block)

    # We must receive a reward for finding the proof.
    # The sender is "0" to signify that this node has mined a new coin.
    blockchain.new_transaction(
        sender="0",
        recipient=node_identifier,
        amount=1,
    )

    # Forge the new Block by adding it to the chain
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)

    response = {
        'message': "New Block Forged",
        'index': block['index'],
        'transactions': block['transactions'],
        'proof': block['proof'],
        'previous_hash': block['previous_hash'],
    }
    return jsonify(response), 200

#### @app.route('/transactions/new', methods=['POST'])
- `/transactions/new`로 요청하면, 새 트랙잭션을 추가하도록 한다.
- 이 때, user로 부터 sender, recipient, amount정보를 모두 받아왔는지 확인한 후,
- 받아온 정보를 위의 new_transaction(self, sender, recipient, amount) 함수에서 처리하도록 한다.

In [ ]:
@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json()

    # Check that the required fields are in the POST'ed data
    required = ['sender', 'recipient', 'amount']
    if not all(k in values for k in required):
        return 'Missing values', 400

    # Create a new Transaction
    index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount'])

    response = {'message': f'Transaction will be added to Block {index}'}
    return jsonify(response), 201

#### @app.route('/chain', methods=['GET'])
-`/chain`로 요청하면, 현 체인의 길이를 json형태로 보여준다.

In [ ]:
@app.route('/chain', methods=['GET'])
def full_chain():
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain),
    }
    return jsonify(response), 200

#### @app.route('/nodes/register', methods=['POST']) def register_nodes():
- 새 노드들을 [nodes : "주소"]의 형태로 json을 post형식으로 보내면, 해당 노드들을 추가해준다.

In [ ]:
'''
새 노드들을 [nodes : "주소"]의 형태로 json을 post형식으로 보내면, 해당 노드들을 추가해준다.
'''
@app.route('/nodes/register', methods=['POST'])
def register_nodes():
    values = request.get_json()

    nodes = values.get('nodes')
    if nodes is None:
        return "Error: Please supply a valid list of nodes", 400

    for node in nodes:
        blockchain.register_node(node)

    response = {
        'message': 'New nodes have been added',
        'total_nodes': list(blockchain.nodes),
    }
    return jsonify(response), 201

#### @app.route('/nodes/resolve', methods=['GET']) def consensus():
- '/nodes/resolve'로 요청하면, 체인 분기로 인한 충돌을 병합해준다. 지금 노드의 체인이 더 길면 살아남고, 짧으면 대체된다.

In [ ]:
'''
'/nodes/resolve'로 요청하면, 체인 분기로 인한 충돌을 병합해준다.
지금 노드의 체인이 더 길면 살아남고, 짧으면 대체된다.

'''
@app.route('/nodes/resolve', methods=['GET'])
def consensus():
    replaced = blockchain.resolve_conflicts()

    if replaced:
        response = {
            'message': 'Our chain was replaced',
            'new_chain': blockchain.chain
        }
    else:
        response = {
            'message': 'Our chain is authoritative',
            'chain': blockchain.chain
        }

    return jsonify(response), 200

In [21]:
if __name__ == '__main__':
    from argparse import ArgumentParser

    parser = ArgumentParser()
    parser.add_argument('-p', '--port', default=5000, type=int, help='port to listen on')
    args = parser.parse_args()
    port = args.port

    app.run(host='0.0.0.0', port=port)

NameError: name 'app' is not defined